In [ ]:
#from shrink_dataset import extract_datasets
#extract_datasets()

In [1]:
import wandb
from evaluate_task import Task
GPT3_5 = "gpt-3.5-turbo-0125" #metti la preview
GPT4 = "gpt-4-0125-preview"

anachronism = Task('multiple_choice',
                   'datasets/task_anachronisms.json',
                   'Does the following sentence contain non-contemporaneous (anachronistic) elements?.')
logical_sequence = Task('multiple_choice',
                        'datasets/task_logical_sequence.json',
                        "Identify the correct chronological or sequential order of items in a list.")
color_HCL = Task('multiple_choice', 'datasets/task_color_HCL.json',"")
color_RGB = Task('multiple_choice', 'datasets/task_color_RGB.json',"")
color_HEX = Task('multiple_choice', 'datasets/task_color_HEX.json',"")
color_HSL = Task('multiple_choice', 'datasets/task_color_HSL.json',"")
matrixshapes = Task('free_response',
                    'datasets/task_matrixshapes.json',
                    "Compute the final matrix shape after the given operations are performed.")
operators = Task('free_response',
                    'datasets/task_operators.json',
                    "Given the definition of the op operator, compute the result.")


In [ ]:
#BASELINE

MULTIPLE_CHOICE_SUFFIX = "Just repeat the answer you choose."
FREE_RESPONSE_SUFFIX = "Please don't show me the steps, i just want to see the final result in one single line."

In [ ]:
#WITH EXPLANATION

MULTIPLE_CHOICE_SUFFIX = "In the first line you answer repeating the answer you choose, in the second line explain your answer in 20 words."
FREE_RESPONSE_SUFFIX = "In the first line you answer just the result, in the second line explain your answer in 20 words."


In [2]:
#CHAIN OF THOUGHTS

MULTIPLE_CHOICE_SUFFIX = "Split the problem in steps, resolve the steps and then in the last line you just repeat the answer you choose."
FREE_RESPONSE_SUFFIX = "Split the problem in steps, resolve the steps and then in the last line answer just the result."

for task in [anachronism, logical_sequence, color_HCL, color_RGB, color_HEX, color_HSL, matrixshapes, operators]:

    wandb.init(project="LLMs-Still-Can-t",config={"model": GPT4,"shots": 0})

    suffix = MULTIPLE_CHOICE_SUFFIX if task.type == 'multiple_choice' else FREE_RESPONSE_SUFFIX
    acc , df = task.evaluate(wandb.config.model, suffix, wandb.config.shots)

    wandb.log({ "accuracy": acc, "examples": wandb.Table(dataframe=df)})

    wandb.run.summary["system_prompt"] = task.system_prompt + suffix
    wandb.run.summary["model"] = wandb.config.model
    wandb.run.summary["shots"] = wandb.config.shots
    wandb.run.summary["name"] = f"ChainOfThoughts [{task.json_string}]"

    wandb.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: s-bresciani026 (simone-bresciani). Use `wandb login --relogin` to force relogin


accuracy,▁
accuracy,0.82
model,gpt-4-0125-preview
name,ChainOfThoughts [dat...
shots,0
system_prompt,Does the following s...


accuracy,▁
accuracy,0.88
model,gpt-4-0125-preview
name,ChainOfThoughts [dat...
shots,0
system_prompt,Identify the correct...


accuracy,▁
accuracy,0.53
model,gpt-4-0125-preview
name,ChainOfThoughts [dat...
shots,0
system_prompt,Split the problem in...


accuracy,▁
accuracy,0.6
model,gpt-4-0125-preview
name,ChainOfThoughts [dat...
shots,0
system_prompt,Split the problem in...


accuracy,▁
accuracy,0.8
model,gpt-4-0125-preview
name,ChainOfThoughts [dat...
shots,0
system_prompt,Split the problem in...


accuracy,▁
accuracy,0.67
model,gpt-4-0125-preview
name,ChainOfThoughts [dat...
shots,0
system_prompt,Split the problem in...


accuracy,▁
accuracy,0.78
model,gpt-4-0125-preview
name,ChainOfThoughts [dat...
shots,0
system_prompt,Compute the final ma...


Completion Exception in example:  {'input': 'op i is the ith power of 10.\nop 2', 'target': '10'}


In [ ]:
#baseline
#explain
#chain of thoughts (che valga per tutte le task)
#few-shots(1 e 5)